<a href="https://colab.research.google.com/github/dsivov/nebula_playground/blob/main/notebooks/nebula_upload_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -U gradient
!pip install ipywidgets
Please change to localhos if running 74.82.29.209 machine
!pip install --index-url http://74.82.29.209:8090 nebula3_database==0.2.2 --trusted-host 74.82.29.209


API KEY - Gradient API KEY to work with Nebula Team
Storage Provider - shared storage for all Gradient Notebooks and WorkFlows

In [ ]:
from gradient import DatasetsClient, DatasetVersionsClient, WorkflowsClient
import ipywidgets as widgets
from IPython.display import display
import yaml
import requests
import uuid
from database.arangodb import DatabaseConnector 

api_key='fdcd43409325ce4d47e6dc1aa911df'
storage_provider_id='splt47zy852oxcg'

datasets_client = DatasetsClient(api_key)
db = DatabaseConnector()


If you run this notebook locally - please change to your dataset directory with dataset movies/images
If you run it from Google Colab - first upload your files (Files -> Create directory -> Upload) 
let's collect some inputs from you:)

In [ ]:
_local_dataset_dir = widgets.Text(
    value='sample_data/',
    placeholder='Type something',
    description='Local Dir:',
    disabled=False
)

_dataset_name = widgets.Text(
    value='new_dataset_01',
    placeholder='Type something',
    description='Dataset:',
    disabled=False
)

_database_name = widgets.Text(
    value='prodemo',
    placeholder='Type something',
    description='DB name:',
    disabled=False
)
_input_type = widgets.Select(
    options=['movie', 'image'],
    value='movie',
    # rows=10,
    description='Type:',
    disabled=False
)

display(_local_dataset_dir)
display(_dataset_name)
display(_database_name)
display(_input_type)

Text(value='/datasets/dima_dataset/', description='Local Dir:', placeholder='Type something')

Text(value='new_dataset_01', description='Dataset:', placeholder='Type something')

Text(value='prodemo', description='DB name:', placeholder='Type something')

Select(description='Type:', options=('movie', 'image'), value='movie')

In [ ]:
print("Use following options: ")
print("    Dataset name: ", _dataset_name.value)
print("    Local files path: ", _local_dataset_dir.value)
print("    DataBase name: ", _database_name.value)
print("    Input type: ", _input_type.value)

local_dataset_dir = _local_dataset_dir.value
dataset_name = _dataset_name.value
database_name = _database_name.value
input_type = _input_type.value


Use following options: 
    Dataset name:  new_dataset_01
    Local files path:  /datasets/dima_dataset/
    DataBase name:  prodemo
    Input type:  image


Create new storage in Gradient Datasets Storage Provider

In [ ]:
dataset_id = datasets_client.create(
    name=dataset_name,
    storage_provider_id=storage_provider_id
)
#Create version
datasetVersions_client = DatasetVersionsClient(api_key)

version_id = datasetVersions_client.create(
    dataset_id=dataset_id
)

In [ ]:
put_command = "gradient datasets files put --id " + dataset_id + ":" + version_id + " --source-path " + local_dataset_dir + " --apiKey " + api_key
commit_command = "gradient datasets versions commit --id "  + dataset_id + ":" + version_id + " --apiKey " + api_key
!{put_command}
!{commit_command}

Update pipelynes entry in database

In [ ]:
wf_template = {
  "movies": {},
  "tasks": {},
  "inputs": {
    "videoprocessing": {
      "is_async": 'false',
      "movies": "/inputs/data",
      "dataset_id": dataset_name,
      "output": "db",
      "input": "dataset",
      "overwrite": 'true',
      "save_movies": 'true'
    }
  }
}
pipeline_entry = wf_template
pipeline_entry['inputs']['videoprocessing']['movies'] =  {'path':'/inputs/data', 'type': input_type}   
pipeline_entry['id'] = str(uuid.uuid4())
pipeline_entry['_key'] = pipeline_entry['id']
print(pipeline_entry)
my_db = db.connect_db(database_name)
my_db.collection("pipelines").insert(pipeline_entry)


{'movies': {}, 'tasks': {}, 'inputs': {'videoprocessing': {'is_async': 'false', 'movies': {'path': '/inputs/data', 'type': 'image'}, 'dataset_id': 'new_dataset_01', 'output': 'db', 'input': 'dataset', 'overwrite': 'true', 'save_movies': 'true'}}, 'id': '8787d408-4e73-4213-8112-c3c5a4a3edce', '_key': '8787d408-4e73-4213-8112-c3c5a4a3edce'}


{'_id': 'pipelines/8787d408-4e73-4213-8112-c3c5a4a3edce',
 '_key': '8787d408-4e73-4213-8112-c3c5a4a3edce',
 '_rev': '_fFpHZAi---'}

Upload files to Gradient dataset storage

Done with dataset upload, lets create workflow definition 


In [ ]:

final_yaml_dictionary = {}
yamldct = {}
#Read the yml from GitHub
URL = "https://raw.githubusercontent.com/NEBULA3PR0JECT/nebula3_pipeline/main/upload_workflow.yaml"
response = requests.get(URL)
yaml_content = response.text
#print(yaml_content)
yamldct = yaml.safe_load(yaml_content)
yamldct['defaults']['env']['ARANGO_DB']=database_name
yamldct['defaults']['env']['PIPELINE_ID']=pipeline_entry['id']
yamldct['defaults']['env']['DATASET_NAME']=dataset_name
yamldct['defaults']['env']['DATASET_PATH']='/inputs/data/'
#DATASET_PATH: /inputs/data
final_yaml_dictionary.update({'inputs': {'data': {'type':'dataset', 'with': {'ref': dataset_id + ":" + version_id}}}})
#print (final_yaml_dictionary)
final_yaml_dictionary.update(yamldct)
#print (final_yaml_dictionary)
#yamldct['inputs']['data']['type']['with']['ref']=dataset_name
print(yamldct)
yaml_to_use = yaml.dump(final_yaml_dictionary, sort_keys=False)
print(yaml_to_use)

{'defaults': {'env': {'PIPELINE_ID': '789957e0-a0e5-4349-bef4-7efdc7d9773c', 'REID_RESULT_PATH': '/datasets/media/services', 'ARANGO_DB': 'dima_test', 'ARANGO_HOST': '172.83.9.249', 'DATASET_NAME': 'new_dataset_01', 'DATASET_PATH': '/inputs/data/'}}, 'jobs': {'videoprocessing': {'inputs': {'data': 'workflow.inputs.data'}, 'resources': {'instance-type': 'RTX4000'}, 'uses': 'container@v1', 'env': {'EXPERT_RUN_MODE': 'task'}, 'with': {'image': 'aharonamir/nebula3_videoprocessing:9ec6a70'}}}}
inputs:
  data:
    type: dataset
    with:
      ref: ds3a6z0eyiqex8a:weacycq
defaults:
  env:
    PIPELINE_ID: 789957e0-a0e5-4349-bef4-7efdc7d9773c
    REID_RESULT_PATH: /datasets/media/services
    ARANGO_DB: dima_test
    ARANGO_HOST: 172.83.9.249
    DATASET_NAME: new_dataset_01
    DATASET_PATH: /inputs/data/
jobs:
  videoprocessing:
    inputs:
      data: workflow.inputs.data
    resources:
      instance-type: RTX4000
    uses: container@v1
    env:
      EXPERT_RUN_MODE: task
    with:
     

In [ ]:
gradient_key = '5d88bfa5909b30076829c101624d67'
workflow_client = WorkflowsClient(gradient_key)
print(yamldct)
#893f5ef9-e652-4e93-97f3-9b65f62293f8
intermediate_value = workflow_client.run_workflow(
    workflow_id='893f5ef9-e652-4e93-97f3-9b65f62293f8',
    spec=final_yaml_dictionary,
    inputs=None
    )


In [ ]:
#datasets_client.delete(
#    dataset_id=dataset_id,
#)